<a href="https://colab.research.google.com/github/VenkteshV/Question_duplicate_detection/blob/main/Week4/Cleaning_JEE_data_from_numericals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code heavily based on the Notebook from Week3 : https://colab.research.google.com/drive/1EyJZbbbjoSxHvqJtTxJBAtiP9S6CE7FL?authuser=1

### Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from string import punctuation
import torch
import pandas as pd
from time import time
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
nltk.download("")

[nltk_data] Error loading : Package '' not found in index


False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tf.test.gpu_device_name()

''

### Loading the data

In [ ]:
df = pd.read_csv("drive/MyDrive/Extramarks Project/Dataset versions/EM-Data-with-answers.csv", index_col = 0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df

,question_id,question,syllabus_id,answer,question_type
0,2073394,Total number of species that can be oxidized b...,698013,All the species which are not in their highest...,Subjective Question
1,2073385,How many following Ammonium salts will evolve ...,698013,Ammonium salts having anions NO2 NO3 ClO4 and ...,Subjective Question
2,2073373,There are some species given below i O2 ii CO ...,697895,4 Molecular orbital configuration for upto 14 ...,Subjective Question
3,2073366,Consider the following compounds 1 H3CF 2 H2CF...,697895,5 stryde images 2022 04 11 2022041195766400164...,Subjective Question
4,2073360,Find total number of reagents which can produc...,698013,Conc H2SO4 2KI K2SO4 HI 12I2 Hg NO3 2 2KI HgI2...,Subjective Question
...,...,...,...,...,...
276057,420544,A dice is rolled thrice The probability of get...,-1,frac 5 24,Single Choice Question
276058,420543,subjective english question,-1,answer_subjective,Subjective Question
276059,415580,question subjective english,-1,answer subjective hindi,Subjective Question
276060,415579,subjective english questions,-1,answer of this question,Subjective Question


In [ ]:
len1 =len(df)
df = df.dropna()
print("Rows with NaN values: ", len1-len(df))

Rows with NaN values:  1846


In [ ]:
df.head(2)

,question_id,question,syllabus_id,answer,question_type
0,2073394,Total number of species that can be oxidized b...,698013,All the species which are not in their highest...,Subjective Question
1,2073385,How many following Ammonium salts will evolve ...,698013,Ammonium salts having anions NO2 NO3 ClO4 and ...,Subjective Question


In [ ]:
print(df.sample(1).iloc[0]['question'])
print(df.sample(1).iloc[0]['question'])
print(df.sample(1).iloc[0]['question'])
print(df.sample(1).iloc[0]['question'])
print(df.sample(1).iloc[0]['question'])

The 12 Alwars sang the glory of 
न म नल ख त क न स च ह न व स मय द ब धक च ह न ह 
Why is it not recommended to measure the temperature of boiling water using a clinical thermometer 
क स प र रम भ ह न व ल उपव क य क न स आश र त उपव क य ह त ह 
Put the verbs in brackets into the simple past or past continuous tense 1 This bull normally ______ not chase people unless something ______ make him angry 2 Kelly said that she ______ not like her present flat and _____ try to find another 3 John _____ find a ring as he ______ dig in the garden 4 He just ____ start when I ______ see it so I couldn t judge 5 While he ______ water the plants in the garden it _____ begin to rain 


######Few samples

In [ ]:
df.iloc[-200:-190]

,question_id,question,syllabus_id,answer,question_type
275860,414979,The adjective which forms an adverb with ily i...,3623,angry,Single Choice Question
275861,414978,The adverb indispensably means _______________,3623,importantly,Single Choice Question
275862,414976,Birds fly ________ fish swim,3632,and,Single Choice Question
275863,414975,Choose the appropriate conjunction You will pa...,3632,if,Single Choice Question
275864,414974,Choose the appropriate conjunction Take care o...,3632,before,Single Choice Question
275865,414933,Choose the appropriate conjunction Answer the ...,3632,before,Single Choice Question
275866,414932,Choose the appropriate conjunction I returned ...,3632,after,Single Choice Question
275867,414931,Choose the appropriate conjunction Many things...,3632,since,Single Choice Question
275868,414930,Choose the appropriate conjunction I am so tir...,3632,that,Single Choice Question
275869,414929,Choose the appropriate conjunction Don t waste...,3632,lest,Single Choice Question


In [ ]:
df.iloc[-300:-290]

,question_id,question,syllabus_id,answer,question_type
275760,416272,Bharat was ___________________ for the job,4415,perfect,Single Choice Question
275761,416271,The population of China is greater ___________...,4415,than that of,Single Choice Question
275762,416270,Either you or I ___________________,4415,am mistaken,Single Choice Question
275763,416269,The collective noun choir means ___________,3637,a group of singers,Single Choice Question
275764,416268,The proper noun in the sentence is ___________...,3637,Kanpur,Single Choice Question
275765,416267,The correct collective noun in the following i...,3637,a pride of peacocks,Single Choice Question
275766,416266,John has hatred for swimming The abstract noun...,3637,hatred,Single Choice Question
275767,416265,The correct adjective order is _______________,3620,a hardworking short forty year old Punjabi dri...,Single Choice Question
275768,416264,We are a good team as ____________________ our...,3636,each of,Single Choice Question
275769,416263,Choose the correctly punctuated sentence,3616,Come here Tom and sit down,Single Choice Question


In [ ]:
df.iloc[-500:-490]

,question_id,question,syllabus_id,answer,question_type
275560,418869,The correct superlative degree of the adjectiv...,3620,biggest,Single Choice Question
275561,418572,The rules made for the Buddhist Sangha were wr...,744376,Vinaya Pitaka,Single Choice Question
275562,418571,Buddha's main teaching was based on the fact t...,744376,Life is full of sufferings and unhappiness,Single Choice Question
275563,418570,The founder of Buddhism was,744376,Siddartha,Single Choice Question
275564,418569,The patient had died before ____________,3659,the doctor came,Single Choice Question
275565,418568,Amy was studying while Mollie _______________ ...,3659,was making,Single Choice Question
275566,418567,Last night at 6 PM __________________ dinner,3659,I was eating,Single Choice Question
275567,418566,Last year I,3659,took my exam,Single Choice Question
275568,418565,Isabel rarely ____________ to attend the music...,3659,comes,Single Choice Question
275569,418544,Yesterday my mother and I ______________ to wa...,3659,went,Single Choice Question


### Cleaning the data

In [ ]:
removed_questions = []
questions_left = []

##### If there is a number in the question delete it 

In [ ]:
for i in range(len(df)):
  if i % 20000 == 0:
    print("Index: ", i)
  patternDigit = re.compile(r'.*\d.*')
  row = df.iloc[i]
  question = row['question']
  # try:
  if len(patternDigit.findall(question)) > 0:
    removed_questions.append(row)
  else:
    questions_left.append(row)
  # except:
  #   print("Error at index: ", str(i), ", type: ", str(type(question)), ", question: " , str(question), ", row: " +str(row))
  #   removed_questions.append(row) 

print(len(removed_questions))
print(len(questions_left))

Index:  0
Index:  20000
Index:  40000
Index:  60000
Index:  80000
Index:  100000
Index:  120000
Index:  140000
Index:  160000
Index:  180000
Index:  200000
Index:  220000
Index:  240000
Index:  260000
116466
157750


Hence we remove 42% of the questions

##### If there are more than 3 numbers concsecutively in the question delete it 

In [ ]:
# removed_questions = []
# questions_left = []

In [ ]:
# for i in range(len(df[:100000])):
#   if i % 20000 == 0:
#     print("Index: ", i)
#   patternDigit = re.compile(r'.*\d\d\d.*')
#   row = df.iloc[i]
#   question = row['question']
#   # try:
#   if len(patternDigit.findall(question)) > 0:
#     removed_questions.append(row)
#   else:
#     questions_left.append(row)
#   # except:
#   #   print("Error at index: ", str(i), ", type: ", str(type(question)), ", question: " , str(question), ", row: " +str(row))
#   #   removed_questions.append(row) 

# print(len(removed_questions))
# print(len(questions_left))

In [ ]:
# temp = []
# for i in range(len(df[:100000])):
#   if i % 20000 == 0:
#     print("Index: ", i)
#   patternDigit = re.compile(r'.*\d.*')
#   row = df.iloc[i]
#   question = row['question']
#   # try:
#   if len(patternDigit.findall(question)) > 0:
#     temp.append(row)
#   # except:
#   #   print("Error at index: ", str(i), ", type: ", str(type(question)), ", question: " , str(question), ", row: " +str(row))
#   #   removed_questions.append(row) 

# print(len(temp))

In [ ]:
# pd.DataFrame(temp)

Hence we remove 20% of the questions

##### If there is a non-ASCII charater in the question delete it 
The devanagri chars have the unicod eindexes 0900 - 097F according to [stackoverflow](https://stackoverflow.com/questions/41356013/how-to-detect-if-a-string-contains-hindi-devnagri-in-it-with-character-and-wor)

In [ ]:
cleanDf = pd.DataFrame(questions_left)
questions_left = []

In [ ]:
for i in range(len(cleanDf)):
  if i % 20000 == 0:
    print("Index: ", i)
  patternHindi = re.compile(r'.*[\u0900-\u097F].*') 
  row = cleanDf.iloc[i]
  question = row['question']
  
  if len(patternHindi.findall(question)) > 0:
    removed_questions.append(row)
  else:
    questions_left.append(row)
  

print(len(removed_questions))
print(len(questions_left))

Index:  0
Index:  20000
Index:  40000
Index:  60000
Index:  80000
Index:  100000
Index:  120000
Index:  140000
140366
133850


##### Replacing all element shortcuts with their full forms

In [ ]:
cleanDf = pd.DataFrame(questions_left)
questions_left = []

In [ ]:
zync potassium 

In [ ]:
cleanDf

,question,syllabus_id
7,How many π bonds are present in ferrocene,700295
10,What is covalency of chlorine atom in second e...,698013
25,In solid X atoms goes to corner of the cube an...,700078
28,Calculate value of x y here x and y are the to...,697895
40,In phosphorus acid if X is number of non bondi...,697895
...,...,...
276441,subjective english question,-1
276442,question subjective english,-1
276443,subjective english questions,-1
276445,test,-1


In [ ]:
cleanDf[-20000:]

,question,syllabus_id
245116,A parallel plate condenser with dielectric of ...,4174
245117,The energy required to charge a parallel plate...,4174
245118,To store electric charge we use,4174
245119,One micro microfarad is equal to,4174
245120,After an isolated conducting sphere is given a...,4174
...,...,...
276441,subjective english question,-1
276442,question subjective english,-1
276443,subjective english questions,-1
276445,test,-1


In [ ]:
len(cleanDf)

134756

### Saving as file

In [ ]:
cleanDf.to_csv("drive/MyDrive/Extramarks Project/Dataset versions/JEE-with-answers-without-numericals.csv")